In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from qat.utils.pydrefcyc import HasId, Ref, RefDict, RefList, AutoPopulate
from __future__ import annotations

from typing import Dict, List, Optional

In [3]:
class At(HasId):
    x: int
    
class Bt(HasId):
    x: int
    s: str
    As: RefDict[At]
    Ds: RefList[Dt]

class Ct(HasId):
    x: int
    As: RefDict[At]
    Bs: RefList[Bt]
    someB: Ref[Bt]

class Dt(HasId):
    x: int
    Cs: RefDict[Ct]

In [5]:
class Outer(AutoPopulate):
    A: list[At]
    B: list[Bt]
    C: list[Ct]
    D: list[Dt]

In [7]:
pick = lambda L, size=3: {l.id: l for l in np.random.choice(L, size=size)}
def make_Outer(count=10):
     A = [At(x=i, id='A' + str(i)) for i in range(count)]
     B = [Bt(x=i, id='B' + str(i), As=pick(A,3), s='blah' + str(i), Ds=[]) for i in range(count)]
     C = [Ct(x=i, id='C' + str(i), As=pick(A,3), Bs=list(pick(B,3).values()), someB=list(pick(B,1).values())[0]) for i in range(count)]
     D = [Dt(x=i, id='D' + str(i), Cs=pick(C,3)) for i in range(count)]

     for bt in B:
          bt.Ds = list(pick(D,3).values())

     return Outer(A=A, B=B,C=C,D=D, id='outer')

In [9]:
O1 = make_Outer()

In [11]:
O1.B[0].Ds[0].x

7

In [12]:
print(O1.model_dump_json(indent=2))

{
  "A": [
    {
      "id": "A0",
      "x": 0
    },
    {
      "id": "A1",
      "x": 1
    },
    {
      "id": "A2",
      "x": 2
    },
    {
      "id": "A3",
      "x": 3
    },
    {
      "id": "A4",
      "x": 4
    },
    {
      "id": "A5",
      "x": 5
    },
    {
      "id": "A6",
      "x": 6
    },
    {
      "id": "A7",
      "x": 7
    },
    {
      "id": "A8",
      "x": 8
    },
    {
      "id": "A9",
      "x": 9
    }
  ],
  "B": [
    {
      "id": "B0",
      "x": 0,
      "s": "blah0",
      "As": {
        "A6": "A6",
        "A3": "A3",
        "A9": "A9"
      },
      "Ds": [
        "D7",
        "D5",
        "D8"
      ]
    },
    {
      "id": "B1",
      "x": 1,
      "s": "blah1",
      "As": {
        "A3": "A3",
        "A2": "A2",
        "A4": "A4"
      },
      "Ds": [
        "D6",
        "D2",
        "D5"
      ]
    },
    {
      "id": "B2",
      "x": 2,
      "s": "blah2",
      "As": {
        "A2": "A2",
        "A8": "A8",
    

In [13]:
O2 = Outer(**O1.model_dump())
O2.B[0].Ds[0].x

7

In [14]:
for _ in range(1000):
    O1 = make_Outer()
    blob = O1.model_dump()
    O2 = Outer(**blob)
    blob2 = O2.model_dump()

    assert blob == blob2

In [13]:
for _ in range(1000):
    O1 = make_Outer()
    blob = O1.model_dump()
    O2 = Outer(**blob)

    assert type(list(O2.D[0].Cs.values())[0].Bs[0].Ds[0]) is Dt

In [ ]:
blob = O1.model_dump()


In [15]:
O1 = make_Outer()

In [16]:
%%timeit

blob = O1.model_dump()
O2 = Outer(**blob)

302 μs ± 4.53 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [73]:
for _ in range(100):
    O1 = make_Outer()
    blob = O1.model_dump()
    O2 = Outer(**blob)
    #assert(O2 == O1)